In [1]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("default.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        t1 = time()
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)
        t2 = time()


    print(circuit(params, wires, s_wires, d_wires, hf_state))
    return params
def ee_exact(symbols, geometry, electrons, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(electrons,qubits)
    values =[]
    for t in range(1):
        if shots==0:
            #dev = qml.device("default.qubit", wires=qubits)
            dev = qml.device("default.qubit", wires=qubits)
        else:
            #dev = qml.device("default.qubit", wires=qubits,shots=shots)
            dev = qml.device("default.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                print('occ going in:', occ)
                qml.X(wires=w)

            print('what is hf_State:', hf_state)
            print('Diagonal part hf state:', hf_state)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            print('off Diagonal part hf state:', hf_state)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
        print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values



# Define the molecule
#symbols  = ['H', 'H', 'H', 'H']
#geometry = np.array([[0.0, 0.0,  0.0],
#                     [0.0, 0.0,  2.8345889829],
#                     [0.0, 2.8345889829,  0.0],
#                     [0.0, 2.8345889829,  2.8345889829]
#                     ], requires_grad = False)
#electrons = 4
#orbitals = 8
#in bohr 
r_bohr = 1.8897259886 
symbols  = [ 'H', 'H', 'H','H']
geometry = np.array([[0.0, 0.0,  0.0],
                     [0.0, 0.0,  1.0*r_bohr],
                     [0.0, 0.0,  2.0*r_bohr],
                     [0.0, 0.0,  3.0*r_bohr]], requires_grad = False)
electrons = 4
charge = 0
orbitals = 8
params = gs_exact(symbols,geometry,electrons,charge,max_iter=100)
print('Params are:', params)

#params_exact=params

eig = ee_exact(symbols, geometry, electrons, charge,params)
print('exact eigenvalues:\n', eig)

-1.674898925616193
Params are: [ 1.61894333e-01  3.42993224e-17  1.61645238e-01 -1.25681154e-16
  2.68632130e-16 -1.39525013e-01  6.71716275e-18 -1.38349676e-01
 -4.40695103e-02  1.43711707e-16  2.28782836e-16 -6.21272693e-01
  6.50974588e-02  1.20194653e-16  3.75150978e-01 -3.10815764e-01
  5.73269001e-16  6.29344625e-17 -3.10342787e-01  3.72596305e-01
 -2.02741918e-16  6.16091853e-02  3.62094525e-01 -4.84416310e-17
  1.91744574e-16 -2.22470365e-01]
occ going in: [4, 1, 2, 3]
occ going in: [4, 1, 2, 3]
occ going in: [4, 1, 2, 3]
occ going in: [4, 1, 2, 3]
what is hf_State: [0 0 0 0 0 0 0 0]
Diagonal part hf state: [0 0 0 0 0 0 0 0]
occ going in: [6, 1, 2, 3]
occ going in: [6, 1, 2, 3]
occ going in: [6, 1, 2, 3]
occ going in: [6, 1, 2, 3]
what is hf_State: [0 0 0 0 0 0 0 0]
Diagonal part hf state: [0 0 0 0 0 0 0 0]
occ going in: [0, 5, 2, 3]
occ going in: [0, 5, 2, 3]
occ going in: [0, 5, 2, 3]
occ going in: [0, 5, 2, 3]
what is hf_State: [0 0 0 0 0 0 0 0]
Diagonal part hf state: [0 0 

In [ ]:
[-0.5246155 , -0.1627531 ,  0.49505785]

### 100 shots

In [ ]:
eig1 = ee_exact(symbols, geometry, electrons, charge,params,shots=100)
print('eigenvalue 100shots:\n', np.array(eig1))

In [ ]:
# Calculate the mean for each position
means = np.mean(eig1, axis=0)

error = eig1 - means

# Define the number of columns (26 columns)
num_columns = 26

error_dict = {}

for i in range(num_columns):  
    error_name = f"error_ex{i+1}"  
    error_dict[error_name] = error[:, i]
    print(f"{error_name}: {error_dict[error_name]}")

for i, error_name in enumerate(error_dict):
    error_list = error_dict[error_name]  
        
# Variance
    if len(error_list) > 1:
        variance = sum(e**2 for e in error_list) / len(error_list) 
    
        # Print the variance for the current error list
        print(f"Variance for ex{i+1}: {variance.real}")

### 500 shots

In [ ]:
eig2 = ee_exact(symbols, geometry, electrons, charge,params,shots=500)
print('eigenvalue 500shots:\n', np.array(eig2))

In [ ]:
# Calculate the mean for each position
means = np.mean(eig2, axis=0)

error = eig2 - means

# Define the number of columns (26 columns)
num_columns = 26

error_dict = {}

for i in range(num_columns):  
    error_name = f"error_ex{i+1}"  
    error_dict[error_name] = error[:, i]
    print(f"{error_name}: {error_dict[error_name]}")

for i, error_name in enumerate(error_dict):
    error_list = error_dict[error_name]  
        
# Variance
    if len(error_list) > 1:
        variance = sum(e**2 for e in error_list) / len(error_list) 
    
        # Print the variance for the current error list
        print(f"Variance for ex{i+1}: {variance.real}")

### 1k shots

In [ ]:
eig3 = ee_exact(symbols, geometry, electrons, charge,params,shots=1000)
print('eigenvalue 1kshots:\n', np.array(eig3))

In [ ]:
# Calculate the mean for each position
means = np.mean(eig4, axis=0)

error = eig4 - means

# Define the number of columns (26 columns)
num_columns = 26

error_dict = {}

for i in range(num_columns):  
    error_name = f"error_ex{i+1}"  
    error_dict[error_name] = error[:, i]
    print(f"{error_name}: {error_dict[error_name]}")

for i, error_name in enumerate(error_dict):
    error_list = error_dict[error_name]  
        
# Variance
    if len(error_list) > 1:
        variance = sum(e**2 for e in error_list) / len(error_list) 
    
        # Print the variance for the current error list
        print(f"Variance for ex{i+1}: {variance.real}")

In [ ]:
# Calculate the mean for each position
means = np.mean(eig3, axis=0)

error = eig3 - means

# Define the number of columns (26 columns)
num_columns = 26

error_dict = {}

for i in range(num_columns):  
    error_name = f"error_ex{i+1}"  
    error_dict[error_name] = error[:, i]
    print(f"{error_name}: {error_dict[error_name]}")

for i, error_name in enumerate(error_dict):
    error_list = error_dict[error_name]  
        
# Variance
    if len(error_list) > 1:
        variance = sum(e**2 for e in error_list) / len(error_list) 
    
        # Print the variance for the current error list
        print(f"Variance for ex{i+1}: {variance.real}")

### 10k shots

In [ ]:
eig4 = ee_exact(symbols, geometry, electrons, charge,params,shots=10000)
print('eigenvalue 10kshots:\n', np.array(eig4))

In [ ]:
# Calculate the mean for each position
means = np.mean(eig4, axis=0)

error = eig4 - means

# Define the number of columns (26 columns)
num_columns = 26

error_dict = {}

for i in range(num_columns):  
    error_name = f"error_ex{i+1}"  
    error_dict[error_name] = error[:, i]
    print(f"{error_name}: {error_dict[error_name]}")

for i, error_name in enumerate(error_dict):
    error_list = error_dict[error_name]  
        
# Variance
    if len(error_list) > 1:
        variance = sum(e**2 for e in error_list) / len(error_list) 
    
        # Print the variance for the current error list
        print(f"Variance for ex{i+1}: {variance.real}")